In [1]:
import cv2

In [2]:
class Person:
    def __init__(self, name):
        self.name = name


In [3]:
class Human:
    def __init__(self, location):
        self.location = location
        self.id = None
        self.dishes = []


In [4]:
class Dish:
    def __init__(self, location, bringer):
        self.location = location
        self.bringer = bringer


In [5]:
def attempt_ID(human):
    pass

In [6]:
def get_nearest(obj, obj_list):
    min_dist = float("inf")
    closest = None
    for i in obj_list:
        # IRL we should replace the distance measures with a proper one
        if i.location - obj.location < min_dist:
            min_dist = i.location - obj.location
            closest = i
    return closest, min_dist

In [7]:
unowned_dishes = []
dishes_in_frame = []
humans_in_frame = []
timers = []

In [8]:
def object_enter_frame(obj):
    if isinstance(obj, Human):
        humans_in_frame.append(obj)
        attempt_ID(obj)
        for dish in unowned_dishes:
            obj.dishes.append(dish)
            dish.bringer = obj

    if isinstance(obj, Dish):
        dishes_in_frame.append(obj)
        if len(known_humans) == 0:
            unowned_dishes.append(obj)
        else:
            h = get_nearest(obj, known_humans)
            obj.bringer = h
            h.dishes.append(obj)

In [9]:
def object_leave_frame(obj):
    if isinstance(obj, Human):
        for dish in obj.dishes:
            timers.append((dish, 5)) # 5 is the number of frames before we declare the dish abandoned
        humans_in_frame.remove(obj)
    
    if isinstance(obj, Dish):
        if obj.bringer is None:
            unowned_dishes.remove(obj)
        else:
            obj.bringer.dishes.remove(obj)
        dishes_in_frame.remove(obj)

In [10]:
def per_frame(frame):
    for h in humans_in_frame:
        if h.id is None:
            attempt_ID(h)

    for _ in range(len(timers)):
        d,t = timers.pop(0)
        if t == 0 and d in dishes_in_frame:
            print("Shame! Shame! Shame!", d.bringer.id.name, "left a dirty dish on the counter!")
        else:
            timers.append((d, t-1))
    
    # then do the logic for entering and leaving

In [12]:
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 